In [1]:
!pip install transformers

In [1]:
from transformers import BertTokenizer, BertForSequenceClassification

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

import re
import pandas as pd
import numpy as np
import pickle
import seaborn as sns

from matplotlib import pyplot as plt
from transformers import pipeline, TextClassificationPipeline


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BILAL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
Category_model = BertForSequenceClassification.from_pretrained('Emran/ClinicalBERT_ICD10_Categories')
Category_tokenizer = BertTokenizer.from_pretrained('Emran/ClinicalBERT_ICD10_Categories')


In [3]:
Category_classifier = pipeline('text-classification', model = Category_model, tokenizer = Category_tokenizer)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [7]:
example = '46YR OLD F HTN DM \nS/P LEFT MRM 20-4-19 AT TIH \n\nNOW PRESENTS IN ER FOR CHANGE OF FLAP DRAIN \n\nC/O LEFT ARM PAIN ---INCREASES ON MOVING ARM \nDENIES FEVER \nON VOREN AND OANADOOL, OHG ENTI HTN \nO/E-> CHEST CLEAR \nABD SOFT NON TENDER\nSURGICAL WOUND NO SIGNS OF INFLAMMATION STICTCHES IN PLACE\nFLAP RADIVAC DRAIN--> NIL--> CHANGED LAST NIGHT\n---------------\nDC ON PREVIOUS MEDS\nDRAIN CHANGED IN ER \nFU IN G/S OPD AS PER SCHEDULE \nCONTNUE AR EXERCISES'

In [9]:
(Category_classifier(example))

[{'label': 'R42', 'score': 0.035132892429828644}]

In [4]:
medical_stopwords = stopwords.words("english")


medical_stopwords.extend(['speaking', 'none', 'time', 'flush'])

In [5]:
def process_clinical_note(clinical_note):
    # Define the sections to remove
    sections_to_remove = [
        "Name:",
        "Unit No:",
        "Admission Date:",
        "Discharge Date:",
        "Date of Birth:",
        "Sex:",
        "Service:",
        "Allergies:",
        "Attending:",
        "Past Medical History:",
        "Social History:",
        "Family History:",
        "Vitals:",
        "Pertinent Results:",
        "Medications on Admission:",
        "Discharge Medications:",
        "Discharge Disposition:",
        "Discharge Condition:",
        "Discharge Instructions:",
        "Followup Instructions:"
    ]

    # Split the clinical note into lines
    lines = clinical_note.split('\n')

    # Initialize the processed note
    processed_note = []

    # Flag to exclude lines within unwanted sections
    exclude_section = False

    # Iterate through the lines and filter unwanted sections
    for line in lines:
        if any(section in line for section in sections_to_remove):
            exclude_section = True
        elif line.strip() == "":
            # Empty lines separate sections, so reset the flag
            exclude_section = False

        if not exclude_section:
            processed_note.append(line)

    # Join the lines to create the final note
    final_note = '\n '.join(processed_note)
    
    sections_to_remove = [
        r'chief complaint',
        r'history of present illness',
        r'Major Surgical or Invasive Procedure',
        r'physical exam',
        r'brief hospital course',
        r'Discharge',
        
        r'completed by',
    ]
    
    for pattern in sections_to_remove:
        final_note = re.sub(pattern, '', final_note, flags=re.IGNORECASE)

    # Define patterns to identify negations
    negation_patterns = [
        r'no\s+\w+',
        r'not\s+\w+',
        r'did\s+not\s+have\s+\w+'
    ]
    
    # Filter out sentences with negations
    sentences = [sentence for sentence in final_note.split('\n') if not any(re.search(pattern, sentence, re.IGNORECASE) for pattern in negation_patterns)]

    # Remove keys and special characters
    cleaned_note = re.sub(r'\w+:', '', '\n'.join(sentences), flags=re.IGNORECASE)  # Remove keys (case-insensitive)
    cleaned_note = re.sub(r'[^a-zA-Z\s]', '', cleaned_note)  # Remove special characters
    # Tokenize the note into sentences based on '\n'
    sentences = [sentence.strip() for sentence in cleaned_note.split('\n') if sentence.strip()]

    # Remove stop words and empty sentences
    sentences = [
        ' '.join(word for word in sentence.split() if word.lower() not in medical_stopwords)
        for sentence in sentences
    ]
    sentences = [item for item in sentences if item != '']

    return sentences

In [6]:
example_2 = ' \nName:  ___                   Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___\n \nChief Complaint:\nInpatient Hospice\n \nMajor Surgical or Invasive Procedure:\nnone\n\n \nHistory of Present Illness:\nPatient was admitted to inpatient hospice at ___. For full H&P \nsee prior discharge summary or admission note.\n \nPast Medical History:\nEtOH cirrhosis  \nCalculus of gallbladder without cholecystitis  \nCOPD  \nFatty liver  \nIron deficiency anemia  \nh/o aspiration pneumonitis  \n\nPast Surgical History:\ns/p partial gastrectomy ___ for ulcers\n"revision" of gastric ulcer surgery in ___ \n\n \nSocial History:\n___\nFamily History:\nMother w/ DM and ESRD on HD  \nNo family members with liver disease. \n\n \nPhysical Exam:\nAdmission to inpatient hospice exam:\nPatient lying comfortably in bed, sleeping. Breathing unlabored.\n\nDeath exam:\nPatient not breathing. No breath or heart sounds. No palpable\npulse. No pupillary reflexes. No withdrawal to firm nailbed\npressure.\nTime of death: ___\n \nPertinent Results:\nSee prior discharge summary for full lab results and other \nstudies. No labs or studies performed while patient on inpatient \nhospice.\n \nBrief Hospital Course:\nPatient admitted to inpatient hospice. Pain and agitation \nmedications were titrated. Medications to manage secretions were \nadded. Patient expired on ___ at ___.\n \nMedications on Admission:\nThe Preadmission Medication list is accurate and complete.\n1. Acetaminophen 500 mg PO Q6H:PRN Pain - Mild/Fever \n2. Albuterol 0.083% Neb Soln 1 NEB IH Q6H:PRN Wheezing, SOB \n3. Cepacol (Sore Throat Lozenge) 1 LOZ PO Q2H:PRN Throat \nirritation  \n4. GuaiFENesin ___ mL PO Q6H:PRN Cough  \n5. Haloperidol 0.5-2 mg IV Q4H:PRN delirium \n6. Heparin Flush (10 units/ml) 1 mL IV DAILY and PRN, VIP line \nflush \n7. Heparin Flush (10 units/ml) 2 mL IV DAILY and PRN, line flush \n\n8. HYDROmorphone (Dilaudid) 0.25-0.5 mg IV Q15MIN:PRN \nmoderate-severe pain or respiratory distress \n9. LORazepam 0.5-2 mg IV Q2H:PRN anxiety, nausea (first line) \n10. Midodrine 20 mg PO Q6H \n11. Ondansetron 4 mg IV Q8H:PRN Nausea/Vomiting - Second Line \n12. Polyethylene Glycol 17 g PO DAILY:PRN Constipation - First \nLine \n13. rifAXIMin 550 mg PO BID \n14. Simethicone 40-80 mg PO QID:PRN bloating \n15. Sodium Chloride 0.9%  Flush 10 mL IV DAILY and PRN, line \nflush \n16. Sodium Chloride 0.9%  Flush 10 mL IV DAILY and PRN, line \nflush \n\n \nDischarge Medications:\nnone\n \nDischarge Disposition:\nExpired\n \nDischarge Diagnosis:\n#Renal failure\n#Liver failure\n#VRE bloodstream infection\n#Spontaneous bacterial peritonitis\n#Alcoholic cirrhosis\n\n \nDischarge Condition:\npatient expired\n\n \nDischarge Instructions:\nPatient expired\n \nFollowup Instructions:\n___\n'

example_2

' \nName:  ___                   Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___\n \nChief Complaint:\nInpatient Hospice\n \nMajor Surgical or Invasive Procedure:\nnone\n\n \nHistory of Present Illness:\nPatient was admitted to inpatient hospice at ___. For full H&P \nsee prior discharge summary or admission note.\n \nPast Medical History:\nEtOH cirrhosis  \nCalculus of gallbladder without cholecystitis  \nCOPD  \nFatty liver  \nIron deficiency anemia  \nh/o aspiration pneumonitis  \n\nPast Surgical History:\ns/p partial gastrectomy ___ for ulcers\n"revision" of gastric ulcer surgery in ___ \n\n \nSocial History:\n___\nFamily History:\nMother w/ DM and ESRD on HD  \nNo family members with liver disease. \n\n \nPhysical Exam:\nAdmission to inpatient hospice exam:\nPatient lying comfortably in bed, sleeping. Breathin

In [7]:
diagnosis_sentences = process_clinical_note(example_2)
diagnosis_sentences

['Inpatient Hospice',
 'Patient admitted inpatient hospice full HP',
 'see prior summary admission note',
 'Past Surgical',
 'sp partial gastrectomy ulcers',
 'revision gastric ulcer surgery',
 'Admission inpatient hospice',
 'Patient lying comfortably bed sleeping Breathing unlabored',
 'Death',
 'pressure',
 'Patient admitted inpatient hospice Pain agitation',
 'medications titrated Medications manage secretions',
 'added Patient expired',
 'HYDROmorphone Dilaudid mg IV PRN',
 'moderatesevere pain respiratory distress',
 'LORazepam mg IV PRN anxiety nausea first line',
 'Midodrine mg PO QH',
 'Ondansetron mg IV PRN NauseaVomiting Second Line',
 'Polyethylene Glycol g PO PRN Constipation First',
 'Line',
 'rifAXIMin mg PO BID',
 'Simethicone mg PO PRN bloating',
 'Sodium Chloride mL IV DAILY PRN line',
 'Sodium Chloride mL IV DAILY PRN line',
 'Renal failure',
 'Liver failure',
 'VRE bloodstream infection',
 'Spontaneous bacterial peritonitis',
 'Alcoholic cirrhosis']

In [24]:
predicted_codes = [
    item for item in (Category_classifier(diagnosis_sentences)) if item['score'] >= 0
]
# (Category_classifier(diagnosis_sentences))


# Create a dictionary to store the highest score for each label
unique_labels_dict = {}

# Iterate through predicted codes and update the dictionary with the highest score
for entry in predicted_codes:
    label = entry['label']
    score = entry['score']
    
    if label not in unique_labels_dict or score > unique_labels_dict[label]['score']:
        unique_labels_dict[label] = {'score': score}

# Create a new list with unique labels and their highest scores
unique_predicted_codes = [{'label': label, 'score': entry['score']} for label, entry in unique_labels_dict.items()]

# Print the result
unique_predicted_codes



[{'label': 'Z71', 'score': 0.6353513598442078},
 {'label': 'Z51', 'score': 0.39684420824050903},
 {'label': 'K91', 'score': 0.7322407364845276},
 {'label': 'R06', 'score': 0.2961735725402832},
 {'label': 'P13', 'score': 0.0158985648304224},
 {'label': 'R03', 'score': 0.6112291812896729},
 {'label': 'M54', 'score': 0.17420603334903717},
 {'label': 'R09', 'score': 0.041065897792577744},
 {'label': 'Z76', 'score': 0.020437845960259438},
 {'label': 'T51', 'score': 0.10893483459949493},
 {'label': 'J98', 'score': 0.41547977924346924},
 {'label': 'B07', 'score': 0.34735384583473206},
 {'label': 'E70', 'score': 0.09759749472141266},
 {'label': 'L98', 'score': 0.0581362247467041},
 {'label': 'N30', 'score': 0.09249883145093918},
 {'label': 'N19', 'score': 0.331452339887619},
 {'label': 'K76', 'score': 0.5551985502243042},
 {'label': 'A48', 'score': 0.22778862714767456},
 {'label': 'K65', 'score': 0.9026634693145752},
 {'label': 'K70', 'score': 0.9668816328048706}]

In [27]:
threshold = 0.3
filtered_data = [
    item for item in (Category_classifier(diagnosis_sentences)) if item['score'] >= threshold and item['label']
]


# Create a dictionary to store the highest score for each label
unique_labels_dict = {}

# Iterate through predicted codes and update the dictionary with the highest score
for entry in filtered_data:
    label = entry['label']
    score = entry['score']
    
    if label not in unique_labels_dict or score > unique_labels_dict[label]['score']:
        unique_labels_dict[label] = {'score': score}

# Create a new list with unique labels and their highest scores
unique_predicted_codes = [{'label': label, 'score': entry['score']} for label, entry in unique_labels_dict.items()]

# Print the result
unique_predicted_codes


[{'label': 'Z71', 'score': 0.6353513598442078},
 {'label': 'Z51', 'score': 0.39684420824050903},
 {'label': 'K91', 'score': 0.7322407364845276},
 {'label': 'R03', 'score': 0.6112291812896729},
 {'label': 'J98', 'score': 0.41547977924346924},
 {'label': 'B07', 'score': 0.34735384583473206},
 {'label': 'N19', 'score': 0.331452339887619},
 {'label': 'K76', 'score': 0.5551985502243042},
 {'label': 'K65', 'score': 0.9026634693145752},
 {'label': 'K70', 'score': 0.9668816328048706}]